In [12]:
#-*- coding: utf-8 -*-
#https://github.com/hwalsuklee/tensorflow-generative-model-collections/blob/bbba0131d2b73e216d5de7ae2311eb24eb78eb66/ops.py
from __future__ import division
import os
import time
import tensorflow as tf
from tensorflow.contrib import slim
import numpy as np
import os, gzip
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)


Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [13]:
def load_mnist():
    data_dir = os.path.join("./data")

    def extract_data(filename, num_data, head_size, data_size):
        with gzip.open(filename) as bytestream:
            bytestream.read(head_size)
            buf = bytestream.read(data_size * num_data)
            data = np.frombuffer(buf, dtype=np.uint8).astype(np.float)
        return data

    data = extract_data(data_dir + '/train-images-idx3-ubyte.gz', 60000, 16, 28 * 28)
    trX = data.reshape((60000, 28, 28, 1))

    data = extract_data(data_dir + '/train-labels-idx1-ubyte.gz', 60000, 8, 1)
    trY = data.reshape((60000))

    data = extract_data(data_dir + '/t10k-images-idx3-ubyte.gz', 10000, 16, 28 * 28)
    teX = data.reshape((10000, 28, 28, 1))

    data = extract_data(data_dir + '/t10k-labels-idx1-ubyte.gz', 10000, 8, 1)
    teY = data.reshape((10000))

    trY = np.asarray(trY)
    teY = np.asarray(teY)

    X = np.concatenate((trX, teX), axis=0)
    y = np.concatenate((trY, teY), axis=0).astype(np.int)

    seed = 547
    np.random.seed(seed)
    np.random.shuffle(X)
    np.random.seed(seed)
    np.random.shuffle(y)

    y_vec = np.zeros((len(y), 10), dtype=np.float)
    for i, label in enumerate(y):
        y_vec[i, y[i]] = 1.0

    return X / 255., y_vec


In [22]:
data_X, data_y = load_mnist()
print(data_X.shape)

epoch = 20
batch_size = 100
z_dim = 62         # dimension of noise-vector
c_dim = 1
num_batches = len(data_X) // batch_size
start_epoch = 0
start_batch_id = 0
input_height = 28
input_width = 28
output_height = 28
output_width = 28
learning_rate = 0.0002
beta1 = 0.5
image_dims = [input_height, input_width, c_dim]

(70000, 28, 28, 1)


In [15]:
def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                 tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
        initializer=tf.constant_initializer(bias_start))
        if with_w:
            return tf.matmul(input_, matrix) + bias, matrix, bias
        else:
            return tf.matmul(input_, matrix) + bias

def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

def bn(x, is_training, scope):
    return tf.layers.batch_normalization(x, epsilon = 1e-5, momentum = 0.9, training = is_training, name = scope)

def conv2d(input_, output_dim, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="conv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim], initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')
        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())
        return conv
    
def discriminator(x, is_training=True, reuse=False):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
    with tf.variable_scope("discriminator", reuse=reuse):

        net = lrelu(conv2d(x, 64, 4, 4, 2, 2, name='d_conv1'))
        net = lrelu(bn(conv2d(net, 128, 4, 4, 2, 2, name='d_conv2'), is_training=is_training, scope='d_bn2'))
        net = tf.reshape(net, [batch_size, -1])
        net = lrelu(bn(linear(net, 1024, scope='d_fc3'), is_training=is_training, scope='d_bn3'))
        out_logit = linear(net, 1, scope='d_fc4')
        out = tf.nn.sigmoid(out_logit)

        return out, out_logit, net

def build_model():
    """ Graph Input """
    inputs = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')
    z = tf.placeholder(tf.float32, [batch_size, z_dim], name='z')
    """ Loss Function """
    D_real, D_real_logits, _ = discriminator(inputs, is_training=True, reuse=False)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
    

In [19]:
tf.reset_default_graph() 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

build_model()

In [21]:

for idx in range(start_batch_id, num_batches):
    batch_images = data_X[idx*batch_size:(idx+1)*batch_size]
    batch_z = np.random.uniform(-1, 1, [batch_size, z_dim]).astype(np.float32)
    
    _, summary_str, d_loss = self.sess.run([d_optim, d_sum, d_loss],
                                               feed_dict={inputs: batch_images, z: batch_z})
    
    
    break

NameError: name 'inputs' is not defined